In [1]:
import pandas as pd
merged_df = pd.read_csv('Merged_DatasetV2.csv')
merged_df.head()

<ipython-input-1-d1843afa6d51>:2: DtypeWarning: Columns (7,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv('Merged_DatasetV2.csv')


,Unnamed: 0,id,type,type-id,score,name,address,coordinates,area,life-span,...,aliases,latitude,longitude,city,state,county,country,lifespan_years,status,services_offered
0,0,f9a1fd95-4c6d-43a6-bb31-4ee8d5a44a8b,Studio,05fa6a09-ff92-3d34-bdbb-5141d3c24f38,100,Columbia (CBS) Studio ‘B’,"49 East 52nd Street, Manhattan, New York City","{'latitude': '40.759164', 'longitude': '-73.97...","{'id': '74e50e58-5deb-4b99-93a2-decbb365c07f',...",{'ended': None},...,NaN,40.759164,-73.974123,Test Restaurant,New York,NaN,US,NaN,Active,NaN
1,1,42e9fa7c-a981-44f9-943c-bf39a8c6ce8d,Venue,cd92781a-a73f-30e8-a430-55d7521338db,100,New Theatre Oxford,"George St, Oxford OX1 2AG","{'latitude': '51.753951', 'longitude': '-1.260...","{'id': '33587495-c775-4483-b819-931c4cf438cd',...",{'ended': None},...,NaN,51.753951,-1.260011,Victoria House Hotel,England,Oxfordshire,GB,NaN,Active,NaN
2,2,34e7351d-715d-4b20-a5d6-3c0bd717eb1d,Religious building,a120ca4a-69e9-3c0e-bc09-21cae26eb621,100,Santuario della Beata Vergine di Ariadello,NaN,"{'latitude': '45.316087', 'longitude': '9.8651...","{'id': '686e7b1e-613f-419c-bcc1-1e84df36586a',...","{'begin': '1666', 'ended': None}",...,NaN,45.316087,9.865165,Genivolta,NaN,NaN,IT,359.0,Active,NaN
3,3,0ad32768-514b-410f-9d5a-40ebb0c3373c,Studio,05fa6a09-ff92-3d34-bdbb-5141d3c24f38,100,ROKU-st,〒106-0032 東京都港区六本木 3-4-24 六本木足立ビルB1F,"{'latitude': '35.664122', 'longitude': '139.72...","{'id': 'd9fd27ac-b36f-492b-b598-039ee0dd08c1',...",{'ended': None},...,"[{'sort-name': 'Roku Studio', 'type-id': 'fb68...",35.664122,139.729426,Tenso Jinja,NaN,NaN,JP,NaN,Active,NaN
4,4,0b050655-48da-4be7-b5de-7a3ee7370b9e,Venue,cd92781a-a73f-30e8-a430-55d7521338db,100,Rätschenmühle,"Schlachthausstraße 22, 73312 Geislingen/Steige","{'latitude': '48.627518', 'longitude': '9.8256...","{'id': '4518c94c-b6ee-40d0-b4b6-fc27b06fa011',...",{'ended': None},...,NaN,48.627518,9.825691,Geislingen an der Steige,NaN,NaN,DE,NaN,Active,NaN


In [2]:
# Drop rows where 'services_offered' is null
merged_df = merged_df.dropna(subset=['services_offered'])

# Drop the specified columns
merged_df = merged_df.drop(columns=['Unnamed: 0', 'id', 'name'])

# Display the updated dataframe
merged_df.head()

# Ensure the column is string type and split by commas (handling spaces)
merged_df['services_offered'] = merged_df['services_offered'].astype(str).str.split(r',\s*')

# Explode the dataframe to have one service per row
merged_df = merged_df.explode('services_offered')

# Trim any leading or trailing spaces (just in case)
merged_df['services_offered'] = merged_df['services_offered'].str.strip()

# Drop any empty values that might have been created
merged_df = merged_df[merged_df['services_offered'].notna() & (merged_df['services_offered'] != '')]

# Reset index after exploding
merged_df = merged_df.reset_index(drop=True)

import re

# Function to remove the last word and underscore
def remove_last_part(text):
    return re.sub(r'_[^_]+$', '', text)  # Removes the last underscore and the word after it

# Apply the function to the column
merged_df['services_offered'] = merged_df['services_offered'].astype(str).apply(remove_last_part)

# Display the cleaned column
print(merged_df['services_offered'].unique())

def impute_df(df):
    # Fill non-numeric columns with "Missing"
    cols_to_fill = df.columns.difference(['lifespan_years'])
    df[cols_to_fill] = df[cols_to_fill].fillna("Missing")

    # Create missing indicator for 'lifespan_years'
    df['lifespan_years_missing'] = df['lifespan_years'].isnull().astype(int)

    # Impute missing values in 'lifespan_years' with the median value
    median_val = df['lifespan_years'].median()
    df['lifespan_years'] = df['lifespan_years'].fillna(median_val)

    return df


['recorded at' 'engineered at' 'edited at' 'mixed at' 'video shot at'
 'lyrics written at' 'produced at' 'premiere' 'written at' 'commissioned'
 'composed at' 'arranged at' 'dedication' 'revised at' 'remixed at']


In [3]:
from sklearn.model_selection import train_test_split


train_df, test_df = train_test_split(merged_df, test_size=0.2, random_state=42)


train_df = impute_df(train_df)
test_df = impute_df(test_df)

In [4]:
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

# Assume train_df and test_df are already loaded.

# Transform your training data as before.
X_train = pd.get_dummies(train_df.drop(columns=["services_offered", "lifespan_years"]), drop_first=True)
X_train["lifespan_years"] = train_df["lifespan_years"]
X_train.columns = X_train.columns.astype(str).str.replace(r"[,\[\]<>]", "_", regex=True)

# Similarly, transform the test data.
X_test = pd.get_dummies(test_df.drop(columns=["services_offered", "lifespan_years"]), drop_first=True)
X_test["lifespan_years"] = test_df["lifespan_years"]
X_test.columns = X_test.columns.astype(str).str.replace(r"[,\[\]<>]", "_", regex=True)

# Reindex X_test to have the same columns as X_train, filling missing columns with zeros.
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Encode the target variable.
le = LabelEncoder()
y_train = le.fit_transform(train_df["services_offered"])
y_test = le.fit_transform(test_df["services_offered"])

In [8]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate

best_params = {
    "learning_rate": 0.2,
    "max_depth": 3,
    "n_estimators": 100,
    "subsample": 1.0,
    "colsample_bytree": 1.0,
    "random_state": 42
}

final_model = xgb.XGBClassifier(**best_params)

final_model.fit(X_train, y_train, verbose=False)

y_pred = final_model.predict(X_test)

# Evaluate model
f1 = f1_score(y_test, y_pred, average="macro")
print(f"Final Model Macro F1 Score on Test Set: {f1:.4f}")

Final Model Macro F1 Score on Test Set: 0.0780


ABLATION TESTING ITERATIVELY

In [10]:
train_df.columns

Index(['type', 'type-id', 'score', 'address', 'coordinates', 'area',
       'life-span', 'disambiguation', 'aliases', 'latitude', 'longitude',
       'city', 'state', 'county', 'country', 'lifespan_years', 'status',
       'services_offered', 'lifespan_years_missing'],
      dtype='object')

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

base_features = [
    "type", "type-id", "score", "coordinates", "area", "aliases", "latitude", "longitude",
    "city", "state", "county", "country", "lifespan_years", "status"
]

# Convert categorical features to one-hot encoding
X_train = pd.get_dummies(train_df[base_features], drop_first=True)
X_test = pd.get_dummies(test_df[base_features], drop_first=True)

# Align test set columns with training set
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Ensure valid feature names for XGBoost
X_train.columns = X_train.columns.astype(str).str.replace(r"[,\[\]<>]", "_", regex=True)
X_test.columns = X_test.columns.astype(str).str.replace(r"[,\[\]<>]", "_", regex=True)

# Encode target variable
le = LabelEncoder()
y_train = le.fit_transform(train_df["services_offered"])
y_test = le.transform(test_df["services_offered"])

# Define best parameters
best_params = {
    "learning_rate": 0.2,
    "max_depth": 3,
    "n_estimators": 50,
    "subsample": 1.0,
    "colsample_bytree": 1.0,
    "random_state": 42
}

model = xgb.XGBClassifier(**best_params)
model.fit(X_train, y_train, verbose=False)
y_pred = model.predict(X_test)
baseline_f1 = f1_score(y_test, y_pred, average="macro")

print(f"Baseline Model Macro F1 Score: {baseline_f1:.4f}")

# Perform ablation testing by removing one feature at a time
ablation_results = {}

for feature in base_features:
    # Create new dataset without one feature
    reduced_features = [f for f in base_features if f != feature]
    X_train_reduced = pd.get_dummies(train_df[reduced_features], drop_first=True)
    X_test_reduced = pd.get_dummies(test_df[reduced_features], drop_first=True)

    # Align test set with training set
    X_test_reduced = X_test_reduced.reindex(columns=X_train_reduced.columns, fill_value=0)

    # Ensure valid feature names for XGBoost
    X_train_reduced.columns = X_train_reduced.columns.astype(str).str.replace(r"[,\[\]<>]", "_", regex=True)
    X_test_reduced.columns = X_test_reduced.columns.astype(str).str.replace(r"[,\[\]<>]", "_", regex=True)

    # Train the model with the reduced feature set
    model = xgb.XGBClassifier(**best_params)
    model.fit(X_train_reduced, y_train, verbose=False)
    y_pred_reduced = model.predict(X_test_reduced)

    # Compute F1 score
    f1_reduced = f1_score(y_test, y_pred_reduced, average="macro")
    ablation_results[feature] = f1_reduced

    print(f"Removed {feature}: Macro F1 Score = {f1_reduced:.4f} (Δ = {f1_reduced - baseline_f1:.4f})")

# Convert results to a DataFrame for analysis
ablation_df = pd.DataFrame(list(ablation_results.items()), columns=["Feature Removed", "Macro F1 Score"])
ablation_df["F1 Score Difference"] = ablation_df["Macro F1 Score"] - baseline_f1



Baseline Model Macro F1 Score: 0.0849
Removed type: Macro F1 Score = 0.0626 (Δ = -0.0223)
Removed type-id: Macro F1 Score = 0.0846 (Δ = -0.0003)
Removed score: Macro F1 Score = 0.0849 (Δ = 0.0000)
Removed coordinates: Macro F1 Score = 0.0849 (Δ = 0.0000)
Removed area: Macro F1 Score = 0.0767 (Δ = -0.0081)
Removed aliases: Macro F1 Score = 0.0849 (Δ = 0.0000)
Removed latitude: Macro F1 Score = 0.0849 (Δ = 0.0000)
Removed longitude: Macro F1 Score = 0.0849 (Δ = 0.0000)
Removed city: Macro F1 Score = 0.0850 (Δ = 0.0001)
Removed state: Macro F1 Score = 0.0846 (Δ = -0.0003)
Removed county: Macro F1 Score = 0.0849 (Δ = 0.0001)
